## Task List

-Top 10 Happiest Countries (max on happiness) 

Life Ladder = 10 being best, 0 worst


CHARTS/GRAPHS/PLOTS
Avg Region Rank of Happinness
-region - happinness score

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
from pprint import pprint

# import gmaps
# import os

# # Import API key
# from api_keys import g_key

# # # Access maps with unique API key
# gmaps.configure(api_key=g_key)

In [18]:
# Pull CSV Files
happiness_path = "Resources/DataPanelWHR2021C2.csv"
mortality_path = "Resources/MortalityDataWHR2021C2.csv"
regions_path = "Resources/All Countries/all.csv"

# Read CSV - Create Dataframes
happy_df = pd.read_csv(happiness_path, parse_dates=['year'])
happy_df.groupby(['year']).count()


,Country name,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
year,,,,,,,,,,
2005-01-01,27,27,27,27,27,25,1,25,26,26
2006-01-01,89,89,88,89,88,86,81,83,89,88
2007-01-01,102,102,102,100,101,101,102,98,101,102
2008-01-01,110,110,110,110,108,107,109,107,108,110
2009-01-01,114,114,113,113,111,112,113,111,113,113
2010-01-01,124,124,123,120,121,122,123,119,120,120
2011-01-01,146,146,145,145,141,145,144,138,145,146
2012-01-01,142,142,140,142,136,141,133,132,142,142
2013-01-01,137,137,136,137,133,136,133,130,135,137


In [62]:
death_df = pd.read_csv(mortality_path)
#death_df.head()

In [1]:
regions_df = pd.read_csv(regions_path)
regions_df = regions_df.rename(columns={'name': 'Country name'}) 

regions_df


NameError: name 'pd' is not defined